In [ ]:
# Spotify best hits for 90's track music 
# Code 2 concern US, GB, DE, FR, and JP

import requests
import base64

# Client ID and Client Secret
client_id = 'f054229c9b634cbcb2d15c39841b0565'
client_secret = '965028aa060f4aa6a2d1437dd051d6e0'

# Access Token
def get_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_header = base64.b64encode((client_id + ':' + client_secret).encode('ascii')).decode('ascii')
    auth_data = {
        'grant_type': 'client_credentials'
    }
    headers = {
        'Authorization': 'Basic ' + auth_header
    }
    response = requests.post(auth_url, headers=headers, data=auth_data)
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        raise Exception('Could not authenticate, check your client credentials.')

token = get_token(client_id, client_secret)

# Function to search for hits in a specific country
def search_hits_90s_by_country(token, country, limit=10):
    search_url = "https://api.spotify.com/v1/search"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    query = {
        "q": "year:1990-1999",  
        "type": "track",
        "limit": limit,
        "market": country  # Use specific country code
    }
    response = requests.get(search_url, headers=headers, params=query)
    if response.status_code == 200:
        return response.json()['tracks']['items']
    else:
        raise Exception(f"Error fetching data: {response.status_code}")

# List of countries to query (you can combine countries from different continents)
countries = ['US', 'GB', 'DE', 'FR', 'JP']  # US, UK, Germany, France, Japan

# Dictionary to store unique tracks across countries
unique_tracks = {}

# Search for hits in each country and aggregate results
for country in countries:
    print(f"--- Fetching Top 90s Hits in {country} ---")
    tracks = search_hits_90s_by_country(token, country)
    for track in tracks:
        track_id = track['id']
        if track_id not in unique_tracks:
            unique_tracks[track_id] = track

# Convert the dictionary to a list and sort by popularity
sorted_tracks = sorted(unique_tracks.values(), key=lambda x: x['popularity'], reverse=True)

# Print the aggregated and sorted results
print("\n--- Aggregated and Sorted Top 90s Hits Across Countries ---\n")
for idx, track in enumerate(sorted_tracks, start=1):
    print(f"{idx}. {track['name']} by {track['artists'][0]['name']}")
    print(f"Album: {track['album']['name']}")
    print(f"Popularity: {track['popularity']}")
    print(f"Link: {track['external_urls']['spotify']}\n")
